In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import utilities.plotting_utilities
import pandas as pd

from initial_states.parallelclifford import ParallelCliffordLoader
from initial_states.pyramidclifford import PyramidCliffordLoader

In [5]:
n = 8
rank = 1
np.random.seed(0) # fixing the seed for reproducibility
mat = np.random.randn(n,rank)
Q = mat / np.sqrt(np.sum(mat*mat, axis=0)) 
# Q, _ = np.linalg.qr(np.random.randn(n, n)) # for std projection DPP # K = W @ W.T # the DPP kernel # only when Q comes from QR
W = Q[:,:rank]

In [ ]:
circuit = ParallelCliffordLoader(W) # (with FBS gates)
circuit.decompose().draw(
    output="mpl", 
    filename="fig/example_circuit.pdf",
    scale = 1,
    fold=300)

In [ ]:
circuit = PyramidCliffordLoader(W) # (with RBS gates)
circuit.decompose().draw(
    output="mpl", 
    filename="fig/example_circuit.pdf",
    scale = 1,
    fold=300)

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit.library import XGate
from initial_states.utils.givensdecompose import _parity_gate

# n is total nb of qubits
n = 9
register = QuantumRegister(n)
linear_depth_parity_gate = False
reverse = False
a = 0
b = n-1
decomposition = _parity_gate(register, a, b, linear_depth_parity_gate, reverse)
circuit = QuantumCircuit(register)

# planted_parity = 0 # even number of Xs
# circuit.append(XGate(),[register[1]])
# circuit.append(XGate(),[register[4]])
# circuit.append(XGate(),[register[5]])
# circuit.append(XGate(),[register[6]])

# circuit.barrier(register)

for gate, qubits in decomposition:
    circuit.append(gate, qubits)
    
# meas = qk.QuantumCircuit(n, n) # N qubits, N classical bits
# meas.barrier(range(n)) # the barrier is optional, it is an instruction for the later transpiler
# meas.measure(range(n), range(n)) # perform the measurement, record it in the classical bits
# circuit.add_register(meas.cregs[0])
# qc = circuit.compose(meas)
    
circuit.draw(
    output="mpl", 
    filename="fig/example_circuit.pdf",
    scale = 0.9,
    fold=150)

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister
from qiskit_nature.second_q.mappers import QubitMapper
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit.circuit.library import XGate

from initial_states.utils.givensdecompose import _givens_decompose_column_parallel
from initial_states.utils.givensdecompose import _givens_decompose_column_pyramid

n = 16
x = np.zeros((n,1))
x[0] = 0.
x[8] = 1.
x = x / np.sqrt(np.sum(x*x, axis=0)) 

print("x = ", x)
data_matrix = x
n, m = data_matrix.shape
register = QuantumRegister(n)
decomposition_reversed  = _givens_decompose_column_parallel(register, data_matrix)
# decomposition_reversed  = _givens_decompose_column_pyramid(register, data_matrix)
# we build U (c_1 + c_1^*) U^{-1}
qc = QuantumCircuit(register) # Circuit of U
for gate, qubits in decomposition_reversed:
    qc.append(gate, qubits)
    
qc.draw(
output="mpl", 
filename="fig/example_circuit.pdf",
scale = 0.9,
fold=150)